#### Import Section

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import scorer
from sklearn.metrics import accuracy_score

C:\Users\moham\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\moham\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Database connection

In [2]:
conn_string = "host='localhost' dbname='stockmarket' user='postgres' password='5265104d'"
conn = psycopg2.connect(conn_string)

## Read dataset

In [3]:
df = pd.read_sql_query(
    '''select i.date ,i.open ,i.high ,i.low ,i.close ,i.adj_close ,i.volume ,c.name
    from dataset_information i 
    join dataset_company c
    on i.company_id=c.id
    where company_id = {}'''.format(3)  , conn, index_col='date')

In [4]:
print(df.head())

                open      high       low     close  adj_close      volume  \
date                                                                        
1986-03-12  0.444196  0.448661  0.441964  0.441964   0.020108  21420000.0   
1986-03-13  0.441964  0.446429  0.435268  0.441964   0.020108  28991200.0   
1986-03-14  0.441964  0.468750  0.441964  0.466518   0.021225  96213600.0   
1986-03-17  0.464286  0.464286  0.453125  0.464286   0.021123  29680000.0   
1986-03-18  0.464286  0.486607  0.462054  0.479911   0.021834  62339200.0   

            name  
date              
1986-03-12  AAPL  
1986-03-13  AAPL  
1986-03-14  AAPL  
1986-03-17  AAPL  
1986-03-18  AAPL  


## Calculate bolingerband 

In [5]:
df["20d"] = np.round(df["close"].rolling(window = 20, center = False).mean(), 2)
# 2. Compute rolling standard deviation
apple_rstd = np.round(df['close'].rolling(window = 20, center = False).std(), 2)

# 3. Compute upper and lower bands
df['upperband'] = df['20d'] + 2 * apple_rstd
df['lowerband'] = df['20d'] - 2 * apple_rstd

## prepairing dataframe with label

In [6]:
#feature_cols = ['high-upperband', 'low-lowerband','dicesion']

df['high-upperband'] = df['high']-df['upperband']
df['low-lowerband'] =df['low']-df['lowerband']
df['decision'] = np.where(df['high-upperband'] > 0.0 , 'Sell',np.where( df['low-lowerband'] < 0.0,'Buy','Buy & Sell'))
print(df.tail())

                  open        high         low       close   adj_close  \
date                                                                     
2018-03-07  174.940002  175.850006  174.270004  175.029999  175.029999   
2018-03-08  175.479996  177.119995  175.070007  176.940002  176.940002   
2018-03-09  177.960007  180.000000  177.389999  179.979996  179.979996   
2018-03-12  180.289993  182.389999  180.210007  181.720001  181.720001   
2018-03-13  182.589996  183.500000  179.240005  179.970001  179.970001   

                volume  name     20d  upperband  lowerband  high-upperband  \
date                                                                         
2018-03-07  31703500.0  AAPL  170.85     185.69     156.01       -9.839994   
2018-03-08  23774100.0  AAPL  171.72     185.78     157.66       -8.660005   
2018-03-09  32185200.0  AAPL  172.97     185.13     160.81       -5.130000   
2018-03-12  32207100.0  AAPL  174.23     184.19     164.27       -1.800001   
2018-03-13  3

In [7]:
df = df.dropna()
df

,open,high,low,close,adj_close,volume,name,20d,upperband,lowerband,high-upperband,low-lowerband,decision
date,,,,,,,,,,,,,
1986-04-09,0.493304,0.495536,0.479911,0.484375,0.022037,33829600.0,AAPL,0.48,0.52,0.44,-0.024464,0.039911,Buy & Sell
1986-04-10,0.484375,0.488839,0.479911,0.486607,0.022139,27496000.0,AAPL,0.48,0.52,0.44,-0.031161,0.039911,Buy & Sell
1986-04-11,0.486607,0.491071,0.482143,0.482143,0.021936,18916800.0,AAPL,0.49,0.51,0.47,-0.018929,0.012143,Buy & Sell
1986-04-14,0.482143,0.486607,0.477679,0.479911,0.021834,21240800.0,AAPL,0.49,0.51,0.47,-0.023393,0.007679,Buy & Sell
1986-04-15,0.479911,0.491071,0.479911,0.488839,0.022240,32849600.0,AAPL,0.49,0.51,0.47,-0.018929,0.009911,Buy & Sell
1986-04-16,0.488839,0.508929,0.488839,0.504464,0.022951,52707200.0,AAPL,0.49,0.51,0.47,-0.001071,0.018839,Buy & Sell
1986-04-17,0.504464,0.520089,0.500000,0.517857,0.023560,67524800.0,AAPL,0.49,0.51,0.47,0.010089,0.030000,Sell
1986-04-18,0.517857,0.533482,0.513393,0.531250,0.024170,61919200.0,AAPL,0.49,0.51,0.47,0.023482,0.043393,Sell
1986-04-21,0.533482,0.549107,0.533482,0.542411,0.024678,68387200.0,AAPL,0.50,0.54,0.46,0.009107,0.073482,Sell


## Prepairing Feature columns and label columns

In [8]:
feature_cols = ['close','20d','upperband','lowerband']
label_cols = ['decision']
x = df[feature_cols]
y = df[label_cols]
npY = np.array(y)

In [9]:
def extractBolingerFeatures(df):
    feature_cols = ['adj_close','20d','upperband','lowerband']
    label_cols = ['decision']
    x = df[feature_cols]
    y = df[label_cols]
    npY = np.array(y)
    return (x,y,npY)

## prepairing KNN model

In [10]:
def ApplyKnnModel(fetures,label,predictList):
    knn = KNeighborsClassifier(n_neighbors=3,metric='euclidean')
    knn.fit(fetures,label)
    y_predict = str(knn.predict(predictList))
    return y_predict

### test ApplyDecisionTreeModel

In [11]:
# date = '2018-03-07'
adj_close = 175.029999
midband = 170.85
upperband = 185.69
lowerband = 156.01
predictCol = [[175.02999,170.85,185.69,156.01]]
result = ApplyKnnModel(x,y,predictCol)
print(result)

C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


['Buy & Sell']


In [12]:
metric_options =['minkowski','euclidean','manhattan','cityblock']
knn = KNeighborsClassifier(n_neighbors=3,metric='euclidean')

## Split dataset into test and train parts

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)

## basic KNN test

In [14]:
knn.fit(X_train,y_train)
y_predict = knn.predict(X_test)
print(float(accuracy_score(y_test, y_predict)) *100)

C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


86.27329192546583


## Parameter tunning

In [15]:
weight_options =['uniform', 'distance']
metric_options =['minkowski','euclidean','manhattan','cityblock']
k_range = np.array(range(5, 100, 2))
param_grid = dict(n_neighbors=k_range,weights=weight_options,metric=metric_options)
grid = GridSearchCV(KNeighborsClassifier(),param_grid,cv =10,scoring='accuracy')

grid.fit(x,y)

print(grid.best_score_)

print(grid.best_params_)

print(grid.best_estimator_)



IndexError: too many indices for array